In [72]:
class Pool: # DAI/ETH Pool
    def __init__(self, dai: float, eth:float, fee: float = 0.003):
        self.dai = dai
        self.eth = eth
        self.k = dai * eth
        self.fee = fee
        
    def add_liquidity(self, dai: float, eth: float):
        self.dai += dai
        self.eth += eth
        self.k = self.dai * self.eth

    def remove_liquidity(self, dai: float, eth: float):
        self.dai -= dai
        self.eth -= eth
        self.k = self.dai * self.eth
    
    def get_relative_price(self):
        return self.eth / self.dai

def swap(pool: Pool, amount: float, from_token: str, to_token: str) -> float: # Knowing Input Amounts
    # Given INPUT DAI, return RECEIVED ETH (after fees)
    if from_token == 'dai' and to_token == 'eth':
#         print("Swapping " + str(amount) + " DAI to ETH...")
        eth_received = ((1 - pool.fee) * pool.eth * amount)/(pool.dai + (1 - pool.fee) * amount)
        pool.add_liquidity(amount, 0)
        pool.remove_liquidity(0, eth_received)
#         print(str(amount) + " DAI swapped to " + str(eth_received) + " ETH received")
        return eth_received
    
    # Given INPUT ETH, return RECEIVED DAI (after fees)
    elif from_token == 'eth' and to_token == 'dai':
#         print("Swapping " + str(amount) + " ETH to DAI...")
        dai_received = ((1 - pool.fee) * pool.dai * amount)/(pool.eth + (1 - pool.fee) * amount)
        pool.add_liquidity(0, amount)
        pool.remove_liquidity(dai_received, 0)
#         print(str(amount) + " ETH swapped to " + str(dai_received) + " DAI received")
        return dai_received
    # Error message
    else:
        raise ValueError("Invalid Token Pair. :( ")
        
def swap_getInput(pool: Pool, from_token: str, to_token: str, amountOut: float): # Knowing Output needed
    # Given OUTPUT ETH needed, calculate INPUT DAI required
    if from_token == 'dai' and to_token == 'eth':
#         print("To get " + str(amountOut) + " of ETH...")
        dai_needed = (pool.dai * amountOut) / ((1 - pool.fee) * (pool.eth - amountOut))
#         print("You will need " + str(dai_needed) + " DAI")
        return dai_needed
    
    # Given OUTPUT DAI needed, calculate INPUT ETH required
    if from_token == 'eth' and to_token == 'dai':
#         print("To get " + str(amountOut) + " of DAI...")
        eth_needed = (pool.eth * amountOut) / ((1 - pool.fee) * (pool.dai - amountOut))
#         print("You will need " + str(eth_needed) + " ETH")
        return eth_needed

In [73]:
# Find optimal DAI needed for rel_a = rel_b
import sympy as sp # in-built into Python3 existing libraries, no need to pip-install

def optimal_dai(expensive_pool: Pool, cheap_pool: Pool) -> float:
    rel_e = expensive_pool.get_relative_price()
    rel_c = cheap_pool.get_relative_price()
    dai_e, eth_e = expensive_pool.dai, expensive_pool.eth
    dai_c, eth_c = cheap_pool.dai, cheap_pool.eth
    x = sp.symbols('x')
    eq = sp.Eq((eth_c + rel_c * x)/(dai_c - x), (eth_e - rel_e * x)/(dai_e + x))
    # Solve for x
    sol = sp.solve(eq, x)
    # Print the solution
#     print("x =", sol)
    
    valid_solutions = []
    for s in sol:
        if s <= dai_c:
            valid_solutions.append(s)
    # Print the valid solution(s)
    if len(valid_solutions) == 1:
#         print("x =", valid_solutions[0])
        return valid_solutions[0]
    elif len(valid_solutions) > 1:
        print("More than 1 solution found")
    else:
        print("No valid solution found.")

x = optimal_dai(pool_a, pool_b)
print(x)

1755.46725097166


In [74]:
import copy # to allow copying of the original pools without changing their values when calculating arb

In [103]:
def calculate_arbitrage_eth(pool_a: Pool, pool_b: Pool):
    rel_a = pool_a.get_relative_price()
    rel_b = pool_b.get_relative_price()
    
    temp_a = copy.deepcopy(pool_a)
    temp_b = copy.deepcopy(pool_b)
    
    print("\n----------------------------------------------------------------------------------------")
    print("Current relative prices of the 2 pools: " + str(rel_a) + " and " + str(rel_b))
#     print(rel_a, rel_b)
    if rel_a > rel_b: # Buy cheaper DAI from pool_b
        print("\nBuy " + str(optimal_dai(temp_a, temp_b)) + " DAI from pool_b and sell to pool_a")
        eth_req = swap_getInput(temp_b, 'eth', 'dai', optimal_dai(temp_a, temp_b))
        dai_swapped = swap(temp_b, eth_req, 'eth', 'dai')
        eth_earned = swap(temp_a, dai_swapped, 'dai', 'eth')
        print("ETH required for arbitrage: " + str(eth_req))
        print("ETH earned after arbitrage: " + str(eth_earned))
        print("\nProfit after arbitrage in ETH: \n" + str(eth_earned - eth_req))
        print("\nEnding relative prices for both pools: " + str(temp_a.get_relative_price()) + " and " + str(temp_b.get_relative_price()))
        print("+++++++++++++++++++++++++++++++++++++++++")
        return eth_req, eth_earned - eth_req
    elif rel_a < rel_b:
        print("\nBuy " + str(optimal_dai(temp_b, temp_a)) + " DAI from pool_a and sell to pool_b")
        eth_req = swap_getInput(temp_a, 'eth', 'dai', optimal_dai(temp_b, temp_a))
        dai_swapped = swap(temp_a, eth_req, 'eth', 'dai')
        eth_earned = swap(temp_b, dai_swapped, 'dai', 'eth')
        print("ETH required for arbitrage: " + str(eth_req))
        print("ETH earned after arbitrage: " + str(eth_earned))
        print("\nProfit after arbitrage in ETH: \n" + str(eth_earned - eth_req))
        print("\nEnding relative prices for both pools: " + str(temp_a.get_relative_price()) + " and " + str(temp_b.get_relative_price()))
        print("+++++++++++++++++++++++++++++++++++++++++")
        return eth_req, eth_earned - eth_req
    else:
        print("No arbitrage opportunity present :( ")
        
    
    
print(calculate_arbitrage_eth(pool_a, pool_b))
print(calculate_arbitrage_eth(pool_x, pool_y))


----------------------------------------------------------------------------------------
Current relative prices of the 2 pools: 0.0002 and 0.00018

Buy 1755.46725097166 DAI from pool_b and sell to pool_a
ETH required for arbitrage: 0.328467177362181
ETH earned after arbitrage: 0.344019143866326

Profit after arbitrage in ETH: 
0.0155519665041448

Ending relative prices for both pools: 0.000193168793649719 and 0.000193358016873944
+++++++++++++++++++++++++++++++++++++++++
(0.328467177362181, 0.0155519665041448)

----------------------------------------------------------------------------------------
Current relative prices of the 2 pools: 0.00018 and 0.0002

Buy 1755.46725097166 DAI from pool_a and sell to pool_b
ETH required for arbitrage: 0.328467177362181
ETH earned after arbitrage: 0.344019143866326

Profit after arbitrage in ETH: 
0.0155519665041448

Ending relative prices for both pools: 0.000193358016873944 and 0.000193168793649719
+++++++++++++++++++++++++++++++++++++++++
(0.3

In [99]:
pool_a = Pool(100000, 20)
pool_b = Pool(50000, 9)

In [82]:
print(pool_a.get_relative_price())
print(pool_b.get_relative_price())

0.0002
0.00018


In [100]:
pool_x = Pool(50000, 9)
pool_y = Pool(100000, 20)

In [83]:
print(pool_x.get_relative_price())
print(pool_y.get_relative_price())

0.00018
0.0002


In [40]:
y = swap_getInput(pool_b, 'eth', 'dai', 1755.467251)
print(y)

To get 1755.467251 of DAI...
You will need 0.3284671773676765 ETH
0.3284671773676765


In [17]:
swap(pool_b, 0.3284671773676765, 'eth', 'dai')

Swapping 0.3284671773676765 ETH to DAI...
0.3284671773676765 ETH swapped to 1755.467251 DAI received


1755.467251

In [18]:
swap(pool_a, 1755.467251, 'dai', 'eth')

Swapping 1755.467251 DAI to ETH...
1755.467251 DAI swapped to 0.3440191438717838 ETH received


0.3440191438717838

In [19]:
print(pool_a.get_relative_price())
print(pool_b.get_relative_price())

0.0001931687936496115
0.0001933580168741718


In [22]:
print("ETH required for trade: " + str(0.3284671773676765))
print("ETH received back after the 2 trades: " + str(0.3440191438717838))
print("Arbitrage profit in ETH: " + str(0.3440191438717838 - 0.3284671773676765))

ETH required for trade: 0.3284671773676765
ETH received back after the 2 trades: 0.3440191438717838
Arbitrage profit in ETH: 0.015551966504107284


In [23]:
import sympy as sp

In [26]:
x = sp.symbols('x')
eq = sp.Eq((9+0.00018*x)/(50000 - x), (20-0.0002*x)/(100000 + x))

# Solve for x
sol = sp.solve(eq, x)

# Print the solution
print("x =", sol)

x = [1755.46725097166, 2848244.53274903]


In [30]:
type(sol)

list

In [35]:
valid_solutions = []
for s in sol:
    if s <= 50000:
        valid_solutions.append(s)

# Print the valid solution(s)
if valid_solutions:
    print("x =", valid_solutions[0])
else:
    print("No valid solution found.")

x = 1755.46725097166
